In [5]:
import autogen
import os
from dotenv import load_dotenv

_ = load_dotenv()
from autogen.coding import LocalCommandLineCodeExecutor

In [6]:
from autogen import ConversableAgent, AssistantAgent, GroupChat, GroupChatManager

In [7]:
executor = LocalCommandLineCodeExecutor(timeout=500, work_dir="temp")

In [8]:
llm_config = {"model": "gpt-4o-mini"}


In [13]:
train_dir = '../dataset/train'
val_dir = '../dataset/val'
test_dir = '../dataset/test'

In [10]:
def create_data_generators(train_dir, val_dir, test_dir=None):
    """Creates training, validation, and optional test data generators."""
    
    from tensorflow.keras.preprocessing.image import ImageDataGenerator


    IMG_WIDTH, IMG_HEIGHT = 128, 128  
    BATCH_SIZE = 4 

    train_dir = '../dataset/train'
    val_dir = '../dataset/val'
    test_dir = '../dataset/test'
    

    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
    )


    val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)


    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )


    validation_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    test_generator = None
    if test_dir:
        test_generator = val_test_datagen.flow_from_directory(
            test_dir,
            target_size=(IMG_WIDTH, IMG_HEIGHT),
            batch_size=BATCH_SIZE,
            class_mode='categorical'
        )


    return train_generator, validation_generator, test_generator


In [11]:
user_proxy = ConversableAgent(
    name = "Admin",
    system_message = "Given the task, and send instructions to to Project Manager to delegate the task from the message to respective team members.",
    code_execution_config = False,
    llm_config = llm_config,
    human_input_mode = "ALWAYS",
)

In [12]:
project_manager = ConversableAgent(
    name = "project_manager",
    system_message = "Act as a professional Project Manager who excels in creating project plan from a given task, and manages other aspects of project."
    "Given the task, determine"
    "The project plan to achieve the task."
    "After each step is done by other team memmbers, check the progress and instruct the remaining steps."
    "The team members that you manage are, requirement_engineer, system_engineer, software_developer, and tester",
    description = "Project Manager, given the task, determines a project plan to complete the task and delegates it to others. Overseas the other aspects of the project."
    "After each step is done by other team memmbers, check the progress and instruct the remaining steps.",
    llm_config = llm_config, 
)

In [14]:
requirement_engineer = ConversableAgent(
    name = "requirement_engineer",
    system_message = "Act as a professional Requirement Engineer who excels in creating requirement plan from a given task"
    "We need to load data from the correct directories."
    f"The directories are specified: '{train_dir}', '{val_dir}', '{test_dir}'.",
    description = "Requirement Engineer, given the task, determines a requirement plan to complete the task.",
    llm_config = llm_config, 
)

In [16]:
sytem_enginner = ConversableAgent(
    name = "system_engineer",
    llm_config = llm_config,
    system_message = "Act as a professional System Engineer who excels in creating Analysis Plan and Design Plan using the project plan provided by the project manager AND the requirement plan created by the requirement engineer.",
    description = "System Engineer, an engineer who excels in creating analysis and design plan which can easily be converted into codes by software developers."
)

In [19]:
software_developer = AssistantAgent(
    name = "software_developer",
    llm_config = llm_config,
    system_message = "Act as a professional Software Developer who excels in converting design plans received from the System Engineer into Python code. "
        "Follow these guidelines:\n"
        "- Write all code in Python.\n"
        "- Create a very small, computationally light CNN model that runs in a very short time.\n"
        "- Use no more than 4 filters for convolution layers, and limit the Dense layer to 8 neurons.\n"
        "- Limit the training to 3 epochs and ensure the batch size is set to 4 or less for fast execution.\n"
        "- Use input Image size of (128,128) \n"
        "- Set steps_per_epoch to a maximum of 5, and validation steps to 2, to further reduce execution time.\n"
        "- Make sure to set verbose as 0 while fitting evaluating the model."
        "- Do not use pre-trained or complex models.\n"
        "- Ensure that TensorFlow is correctly imported using: from tensorflow.keras.preprocessing.image import ImageDataGenerator. Do not import directly from keras.\n"
        f"- Ensure the CNN model script correctly references the '{train_dir}', '{val_dir}', and '{test_dir}' directories for dataset loading without altering the provided directory paths.\n"
        "- Save the code to a file named cnn_model.py, using markdown code blocks for the output.\n",
        

    description = "Software Developer, an engineer who excels in converting design plans into python codes."
)

In [20]:
tester = ConversableAgent(
    name = "tester",
    system_message = "Execute the python code written by the software developer and report the results back to software developer."
    "Use datagenrator specified by software developer to execute the code."
    "Make sure to following the below guidelines in addition to what software developer says."
    "- First activate virtual environemnt called'venv-autogen' before running the CNN model." 
    "- Make sure to use backslash ('\') while activating virtul environment called venv-autogen."
    "- Run CNN model in venv-autogen environment only."
    "TESTER MUST ALWAYS REPORT BACK TO SOFTWARE DEVELOPER"
    f"Take data for training, validation and testing from following sub directories respectively, training: '{train_dir}', validation: '{val_dir}', testing: '{test_dir}'",
    human_input_mode = "NEVER",
    function_map = { 'create_data_generators': create_data_generators },
    code_execution_config = {
        "executor": executor,
        "last_n_messages": 4,
    },
)

In [21]:
groupchat = GroupChat(
    agents = [user_proxy, project_manager, sytem_enginner, software_developer, tester, requirement_engineer],
    messages = [],
    max_round = 10,
)

In [22]:
manager = GroupChatManager(
    groupchat = groupchat,
    llm_config = llm_config
)

In [23]:
task = "Create a basic CNN deep learning model using keras library for detection of diabetic retinopathy."

In [24]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message = task,
)

Admin (to chat_manager):

Create a basic CNN deep learning model using keras library for detection of diabetic retinopathy.

--------------------------------------------------------------------------------

Next speaker: requirement_engineer


>>>>>>>> USING AUTO REPLY...
requirement_engineer (to chat_manager):

### Requirement Plan for Creating a Basic CNN Deep Learning Model for Detection of Diabetic Retinopathy

#### 1. **Project Overview**
   - The project aims to develop a Convolutional Neural Network (CNN) model using the Keras library for the classification and detection of diabetic retinopathy from retinal images.

#### 2. **Scope of Work**
   - Load and preprocess the datasets from specified directories.
   - Develop, train, and validate a CNN model with appropriate architecture.
   - Evaluate the model's performance using suitable metrics.

#### 3. **Stakeholders**
   - Data Scientists/ML Engineers
   - Quality Assurance Team
   - Project Manager
   - End Users (Healthcare Pr